In [3]:
import math
import numpy as np

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix

import nn
from optim import SGDOptimizer

from supervised_learning import MyMLPClassifier
from dataset.load_data import sklearn_to_df, prepare_data_loader

In [4]:
def prepare_data():
    X, Y = sklearn_to_df(load_digits())
    X_tr, X_te, y_tr, y_te = train_test_split(X, Y, test_size=0.33, random_state=42)
    scaler = MinMaxScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_te = scaler.transform(X_te)
    return X_tr, X_te, y_tr, y_te


def prepare_trainer(model):
    optimizer = SGDOptimizer(model, learning_rate=0.1, regularization=0.01)
    loss_func = nn.CrossEntropyLoss()
    return optimizer, loss_func

def build_model(n_in, n_out):
    np.random.seed(101)
    model = MyMLPClassifier(n_input=n_in, hiddens=[128, 64, 32, 10], n_classes=n_out, activation='leaky_relu')   
    return model

def main():
    X_tr, X_te, y_tr, y_te = prepare_data()
    n_in, n_out = X_tr.shape[1], 10

    num_epochs = 650
    batch_size = 32

    model = build_model(n_in, n_out)
    optimizer, criterion = prepare_trainer(model)

    model.info()

    for epoch in range(num_epochs):
        data_loader = prepare_data_loader(X_tr, y_tr, batch_size)
        step = 0
        total_loss, total_correct = 0, 0
        total_sample = 0

        for batch_X, batch_y in data_loader:
            # Forward pass
            batch_yp = model.forward(batch_X)
            loss = criterion.forward(batch_yp, batch_y)

            # Backward pass and an optimization step
            optimizer.zero_grad()
            dout = criterion.backward()
            model.backward(dout)
            optimizer.step()

            # Log training progress
            step += 1
            total_loss += loss
            total_correct += np.sum(np.argmax(batch_yp, axis=1) == batch_y)
            total_sample += len(batch_y)
        print(f"Epoch: {epoch}, loss={total_loss / total_sample:.4f}, train_acc={total_correct / total_sample:.4f}")

    model.eval()

    ypred = np.argmax(model.forward(X_te), axis=1)
    print("\n Classification report:\n", classification_report(y_te, ypred))


In [5]:
if __name__ == "__main__":
    main()

MyMLPClassifier(
(linear): Linear(in_features=64, out_features=128, bias=True)
(leaky_relu): LeakyReLU()
(linear): Linear(in_features=128, out_features=64, bias=True)
(leaky_relu): LeakyReLU()
(linear): Linear(in_features=64, out_features=32, bias=True)
(leaky_relu): LeakyReLU()
(linear): Linear(in_features=32, out_features=10, bias=True)
)
Epoch: 0, loss=2.2499, train_acc=0.1405
Epoch: 1, loss=2.1920, train_acc=0.2103
Epoch: 2, loss=2.1495, train_acc=0.3192
Epoch: 3, loss=2.1119, train_acc=0.4090
Epoch: 4, loss=2.0759, train_acc=0.4938
Epoch: 5, loss=2.0396, train_acc=0.5503
Epoch: 6, loss=2.0030, train_acc=0.5902
Epoch: 7, loss=1.9665, train_acc=0.6176
Epoch: 8, loss=1.9300, train_acc=0.6351
Epoch: 9, loss=1.8931, train_acc=0.6517
Epoch: 10, loss=1.8554, train_acc=0.6692
Epoch: 11, loss=1.8170, train_acc=0.6825
Epoch: 12, loss=1.7777, train_acc=0.6933
Epoch: 13, loss=1.7376, train_acc=0.6941
Epoch: 14, loss=1.6969, train_acc=0.7024
Epoch: 15, loss=1.6557, train_acc=0.7091
Epoch: 16, 